In [2]:
import pandas as pd
import ast

# Transformaciones Movie_Dataset

In [3]:
movies = pd.read_csv(r'C:\Users\Pedro\Desktop\M-Pindividual\movies_dataset.csv')

C:\Users\Pedro\AppData\Local\Temp\ipykernel_5844\734614192.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(r'C:\Users\Pedro\Desktop\M-Pindividual\movies_dataset.csv')


In [4]:
movies.rename(columns={movies.columns[5]: 'id_Principal'}, inplace=True)
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id_Principal           45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [5]:
print(movies.head)

<bound method NDFrame.head of        adult                              belongs_to_collection    budget  \
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1      False                                                NaN  65000000   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3      False                                                NaN  16000000   
4      False  {'id': 96871, 'name': 'Father of the Bride Col...         0   
...      ...                                                ...       ...   
45461  False                                                NaN         0   
45462  False                                                NaN         0   
45463  False                                                NaN         0   
45464  False                                                NaN         0   
45465  False                                                NaN         0   

                                             

### ***Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.***

In [6]:
#Importamos AST y creamos función para Desempacar

def desempacar_columna(data):
    if pd.isna(data):  # verificamos si hay datos nulos
        return {}
    return ast.literal_eval(data)   #se utiliza para evaluar la cadena de texto data y convertirla en una estructura de datos

In [7]:
# Desempacamos 'belongs_to_collection' y creamos otra tabla

coleccion = movies['belongs_to_collection'].apply(desempacar_columna)

df_coleccion = pd.json_normalize(coleccion)

In [8]:
df_coleccion.head()

,id,name,poster_path,backdrop_path
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,NaN,NaN,NaN,NaN
2,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,NaN,NaN,NaN,NaN
4,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


In [9]:
df_coleccion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4491 non-null   float64
 1   name           4491 non-null   object 
 2   poster_path    3948 non-null   object 
 3   backdrop_path  3263 non-null   object 
dtypes: float64(1), object(3)
memory usage: 1.4+ MB


In [10]:
# Contamos la cantidad de NaN en cada columna

nan_counts = df_coleccion.isna().sum()
print(nan_counts)

id               40975
name             40975
poster_path      41518
backdrop_path    42203
dtype: int64


In [11]:
# Unimos las nuevas columnas al DataFrame original

movies_desempacado = pd.concat([movies, df_coleccion], axis=1)

In [12]:
movies_desempacado.head()

,adult,belongs_to_collection,budget,genres,homepage,id_Principal,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,id,name,poster_path,backdrop_path
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN,NaN,NaN,NaN
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


In [13]:
# Eliminamos la columna original

movies_desempacado.drop(columns=['belongs_to_collection'], inplace=True)

In [14]:
# Desempacamos 'production_companies' y repetimos (creamos otra tabla y luego la unimos a "movies_desempacado"), por último eliminamos la columna original

compania= movies['production_companies'].apply(desempacar_columna)

df_compania = pd.json_normalize(compania)

mov_desempacado = pd.concat([movies_desempacado, df_compania], axis=1)

mov_desempacado.drop(columns=['production_companies'], inplace=True)

mov_desempacado.head()

,adult,budget,genres,homepage,id_Principal,imdb_id,original_language,original_title,overview,popularity,...,16,17,18,19,20,21,22,23,24,25
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,None,None,None,None,None,None,None,None,None,None
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,None,None,None,None,None,None,None,None,None,None
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,None,None,None,None,None,None,None,None,None,None
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,None,None,None,None,None,None,None,None,None,None
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,None,None,None,None,None,None,None,None,None,None


**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

### ***Los valores nulos de los campos 'revenue', 'budget' deben ser rellenados por el número 0.****

In [15]:
#Rellenamos valor nulos en dichas columnas

mov_desempacado[['revenue', 'budget']] = mov_desempacado[['revenue', 'budget']].fillna(0)

print(mov_desempacado[['revenue', 'budget']])

           revenue    budget
0      373554033.0  30000000
1      262797249.0  65000000
2              0.0         0
3       81452156.0  16000000
4       76578911.0         0
...            ...       ...
45461          0.0         0
45462          0.0         0
45463          0.0         0
45464          0.0         0
45465          0.0         0

[45466 rows x 2 columns]


### ***Los valores nulos del campo "release date" deben eliminarse***

In [16]:
movies_df=mov_desempacado.dropna(subset=['release_date'])
print(movies_df[['release_date']])

      release_date
0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
...            ...
45460   1991-05-13
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09

[45379 rows x 1 columns]


In [17]:
print(mov_desempacado[['release_date']])

      release_date
0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
...            ...
45461          NaN
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09

[45466 rows x 1 columns]


### ***De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.***

In [18]:
# Creamos una función para la conversión de Formato Fecha
# Función para convertir fechas a formato AAAA-mm-dd

def formato_date(date):
    try:
        return pd.to_datetime(date).strftime('%Y-%m-%d')
    except ValueError:
        return date

# Aplicar la función a la columna de fechas
movies_df['release_date'] = movies_df['release_date'].apply(formato_date)

C:\Users\Pedro\AppData\Local\Temp\ipykernel_5844\1087990013.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['release_date'] = movies_df['release_date'].apply(formato_date)


In [19]:
# Extraer el año, mes, día de la fecha y trabajamos el posible error donde devuelve NoT (Not a Time) si no es un valor de fecha
movies_df['Anio'] = pd.to_datetime(movies_df['release_date'], errors='coerce').dt.year.fillna(0).astype('int64')
movies_df['Mes'] = pd.to_datetime(movies_df['release_date'], errors='coerce').dt.month.fillna(0).astype('int64')
movies_df['Dia'] = pd.to_datetime(movies_df['release_date'], errors='coerce').dt.day.fillna(0).astype('int64')

C:\Users\Pedro\AppData\Local\Temp\ipykernel_5844\3555970909.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['Anio'] = pd.to_datetime(movies_df['release_date'], errors='coerce').dt.year.fillna(0).astype('int64')
C:\Users\Pedro\AppData\Local\Temp\ipykernel_5844\3555970909.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['Mes'] = pd.to_datetime(movies_df['release_date'], errors='coerce').dt.month.fillna(0).astype('int64')
C:\Users\Pedro\AppData\Local\Temp\ipykernel_5844\3555970909.py

### ***Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.***

In [20]:
#convertir "revenue" y "budget" a float, luego hacer el cálculo

movies_df['revenue'] = pd.to_numeric(movies_df['revenue'], errors='coerce')
movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce')


#cálculo de Retorno de Inversión
movies_df['return'] = movies_df.apply(
    lambda row: row['revenue'] / row['budget'] if pd.notna(row['revenue']) and pd.notna(row['budget']) and row['budget'] != 0 else 0, 
    axis=1
)

print(movies_df[['revenue', 'budget', 'return']])

C:\Users\Pedro\AppData\Local\Temp\ipykernel_5844\4008958657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['revenue'] = pd.to_numeric(movies_df['revenue'], errors='coerce')
C:\Users\Pedro\AppData\Local\Temp\ipykernel_5844\4008958657.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce')


           revenue      budget     return
0      373554033.0  30000000.0  12.451801
1      262797249.0  65000000.0   4.043035
2              0.0         0.0   0.000000
3       81452156.0  16000000.0   5.090760
4       76578911.0         0.0   0.000000
...            ...         ...        ...
45460          0.0         0.0   0.000000
45462          0.0         0.0   0.000000
45463          0.0         0.0   0.000000
45464          0.0         0.0   0.000000
45465          0.0         0.0   0.000000

[45379 rows x 3 columns]


C:\Users\Pedro\AppData\Local\Temp\ipykernel_5844\4008958657.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['return'] = movies_df.apply(


### ***Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.***

In [21]:
df = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
movies_df.drop(columns=df, inplace=True)



C:\Users\Pedro\AppData\Local\Temp\ipykernel_5844\3915960008.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df.drop(columns=df, inplace=True)


In [22]:
# borramos la columna id que sobra, ubicada en la posición 16


movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45379 entries, 0 to 45465
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45376 non-null  float64
 1   genres                45379 non-null  object 
 2   id_Principal          45379 non-null  object 
 3   original_language     45368 non-null  object 
 4   overview              44438 non-null  object 
 5   popularity            45377 non-null  object 
 6   production_countries  45379 non-null  object 
 7   release_date          45379 non-null  object 
 8   revenue               45379 non-null  float64
 9   runtime               45130 non-null  float64
 10  spoken_languages      45376 non-null  object 
 11  status                45296 non-null  object 
 12  tagline               20398 non-null  object 
 13  title                 45376 non-null  object 
 14  vote_average          45376 non-null  float64
 15  vote_count            45

In [23]:

# Cambiar el nombre de la columna 'id' en la posición 2, para que el sistema no se confunda cuando elimine el otro id
movies_df.rename(columns={movies_df.columns[2]: 'id_2'}, inplace=True)


# Lista de índices de las columnas a eliminar
columns_to_drop = [16, 1, 6, 10, 11, 4] + list(range(19, 45))

# Eliminar las columnas usando sus índices
movies_df.drop(columns=movies_df.columns[columns_to_drop], inplace=True)

# Filtrarmos estas filas debido al tamaño del dataframe
movies_df = movies_df.loc[
    (movies_df['original_language'] == 'en') &  
    (movies_df['Anio'].notnull()) &            
    (movies_df['Anio'] >= 1990)]                 


movies_df.info()

C:\Users\Pedro\AppData\Local\Temp\ipykernel_5844\1953193679.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df.rename(columns={movies_df.columns[2]: 'id_2'}, inplace=True)
C:\Users\Pedro\AppData\Local\Temp\ipykernel_5844\1953193679.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df.drop(columns=movies_df.columns[columns_to_drop], inplace=True)


<class 'pandas.core.frame.DataFrame'>
Index: 20968 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             20968 non-null  float64
 1   id_2               20968 non-null  object 
 2   original_language  20968 non-null  object 
 3   popularity         20968 non-null  object 
 4   release_date       20968 non-null  object 
 5   revenue            20968 non-null  float64
 6   runtime            20964 non-null  float64
 7   tagline            11082 non-null  object 
 8   title              20968 non-null  object 
 9   vote_average       20968 non-null  float64
 10  vote_count         20968 non-null  float64
 11  name               2132 non-null   object 
 12  backdrop_path      1670 non-null   object 
 13  Anio               20968 non-null  int64  
 14  Mes                20968 non-null  int64  
 15  Dia                20968 non-null  int64  
 16  return             20968 no

In [24]:
movies_df.rename(columns={movies_df.columns[1]: 'id'}, inplace=True)


movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20968 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             20968 non-null  float64
 1   id                 20968 non-null  object 
 2   original_language  20968 non-null  object 
 3   popularity         20968 non-null  object 
 4   release_date       20968 non-null  object 
 5   revenue            20968 non-null  float64
 6   runtime            20964 non-null  float64
 7   tagline            11082 non-null  object 
 8   title              20968 non-null  object 
 9   vote_average       20968 non-null  float64
 10  vote_count         20968 non-null  float64
 11  name               2132 non-null   object 
 12  backdrop_path      1670 non-null   object 
 13  Anio               20968 non-null  int64  
 14  Mes                20968 non-null  int64  
 15  Dia                20968 non-null  int64  
 16  return             20968 no

In [25]:
 # todos los valores id que no son INT, se reemplaza por 0

# Identificar valores no válidos en la columna 'id' antes de cualquier conversión
invalid_ids_full = movies_df[~movies_df['id'].str.isdigit()]['id'].unique()
movies_df['popularity'] = movies_df['popularity'].astype(float)




In [26]:
# Convertir la columna 'id' a tipo string para manejar todos los valores uniformemente
movies_df['id'] = movies_df['id'].astype(str)

# Identificar valores únicos no válidos (que no son numéricos)
invalid_ids_full = movies_df[~movies_df['id'].str.isdigit()]['id'].unique()

# Mostrar los valores no válidos detectados
invalid_ids_full


array([], dtype=object)

In [27]:

# df=movies_df
# cantidad = list(df.columns).count('id')

# print(cantidad)
#

#movies_df.info()
#Trabajamos los valores con error en id
movies_df['id'] = pd.to_numeric(movies_df['id'], errors='coerce')

In [28]:


movies_df['id'] = movies_df['id'].fillna(0).astype(int)
invalid_count = movies_df['id'].isna().sum()
invalid_count

np.int64(0)

In [31]:
#movies_df['id'] = movies_df['id'].astype(int)
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])

In [32]:

movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20968 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             20968 non-null  float64       
 1   id                 20968 non-null  int64         
 2   original_language  20968 non-null  object        
 3   popularity         20968 non-null  float64       
 4   release_date       20968 non-null  datetime64[ns]
 5   revenue            20968 non-null  float64       
 6   runtime            20964 non-null  float64       
 7   tagline            11082 non-null  object        
 8   title              20968 non-null  object        
 9   vote_average       20968 non-null  float64       
 10  vote_count         20968 non-null  float64       
 11  name               2132 non-null   object        
 12  backdrop_path      1670 non-null   object        
 13  Anio               20968 non-null  int64         
 14  Mes        

In [34]:
movies_df.to_csv("movies.csv", index=False, encoding='utf-8')


# Transformación Credits

In [54]:
credits = pd.read_csv(r'C:\Users\Pedro\Desktop\M-Pindividual\credits.csv')

In [ ]:
# mem_usage_bytes = credits.memory_usage(deep=True).sum()
# print(mem_usage_bytes/1024)
credits.rename(columns={credits.columns[2]: 'id_movies'}, inplace=True)
credits.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   cast       45476 non-null  object
 1   crew       45476 non-null  object
 2   id_movies  45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [ ]:
#Desempacamos Cast y creamos y dataFrame para "CAST"

credits['cast'] = credits['cast'].apply(desempacar_columna)

# Expandir las listas en filas, manteniendo la relación con id_movies
df_cast = credits.explode('cast').reset_index(drop=True)

# Normalizar los diccionarios en columnas separadas, incluyendo id_movies
df_cast_normalizado = pd.concat(
    [df_cast[['id_movies']].reset_index(drop=True),
     pd.json_normalize(df_cast['cast'])],
    axis=1
)

print(df_cast_normalizado)

        id_movies  cast_id                character                 credit_id  \
0             862     14.0            Woody (voice)  52fe4284c3a36847f8024f95   
1             862     15.0   Buzz Lightyear (voice)  52fe4284c3a36847f8024f99   
2             862     16.0  Mr. Potato Head (voice)  52fe4284c3a36847f8024f9d   
3             862     17.0       Slinky Dog (voice)  52fe4284c3a36847f8024fa1   
4             862     18.0              Rex (voice)  52fe4284c3a36847f8024fa5   
...           ...      ...                      ...                       ...   
564887     227506      3.0                           52fe4ea59251416c7515d7d9   
564888     227506      4.0                           52fe4ea59251416c7515d7dd   
564889     227506      5.0                           52fe4ea59251416c7515d7e1   
564890     227506      6.0                           52fe4ea59251416c7515d7e5   
564891     461257      NaN                      NaN                       NaN   

        gender         id  

In [ ]:
df_cast_normalizado['id'] = pd.to_numeric(df_cast_normalizado['id'], errors='coerce').fillna(0).astype(int)
df_cast_normalizado['cast_id'] = pd.to_numeric(df_cast_normalizado['cast_id'], errors='coerce').fillna(0).astype(int)
df_cast_normalizado.head()


,id_movies,cast_id,character,credit_id,gender,id,name,order,profile_path
0,862,14,Woody (voice),52fe4284c3a36847f8024f95,2.0,31,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,862,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,862,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,862,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,862,18,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg


In [ ]:
mem_usage_bytes = df_cast_normalizado.memory_usage(deep=True).sum()
print(mem_usage_bytes/1024)

165721.7529296875


In [ ]:
df_cast_normalizado.drop(columns=['profile_path'], inplace=True)
df_cast_normalizado.head()

,id_movies,cast_id,character,credit_id,gender,id,name,order
0,862,14,Woody (voice),52fe4284c3a36847f8024f95,2.0,31,Tom Hanks,0.0
1,862,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898,Tim Allen,1.0
2,862,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167,Don Rickles,2.0
3,862,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899,Jim Varney,3.0
4,862,18,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900,Wallace Shawn,4.0


In [ ]:
#luego de eliminar profile_path, la composición es la siguiente, eliminamos "credit_id"
#df_cast_normalizado.drop(columns=['credit_id'], inplace=True)
#df_cast_normalizado['id'] = df_cast_normalizado['id'].astype(int)
df_cast_normalizado.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564892 entries, 0 to 564891
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id_movies  564892 non-null  int64  
 1   cast_id    564892 non-null  int64  
 2   character  562474 non-null  object 
 3   credit_id  562474 non-null  object 
 4   gender     562474 non-null  float64
 5   id         564892 non-null  int64  
 6   name       562474 non-null  object 
 7   order      562474 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 34.5+ MB


In [61]:
df_cast_normalizado.to_csv("characters.csv", index=False, encoding='utf-8')

# DataFrame Directores

In [ ]:
# Desenpaquetamos Crew y creamos un DataFrame para Directores

credits['crew'] = credits['crew'].apply(desempacar_columna)

# Expandir las listas en filas, manteniendo la relación con id_movies
df_crew= credits.explode('crew').reset_index(drop=True)

# Normalizar los diccionarios en columnas separadas, incluyendo id_movies
df_crew_normalizado = pd.concat(
    [df_crew[['id_movies']].reset_index(drop=True),
     pd.json_normalize(df_crew['crew'])],
    axis=1
)

df_crew_normalizado.head()

,id_movies,credit_id,department,gender,id,job,name,profile_path
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


In [63]:
# extraemos unicamente 'Director' en trabajo
df_directors = df_crew_normalizado[df_crew_normalizado['job'] == 'Director']


In [64]:
df_directors['id'] = df_directors['id'].astype(int)
df_directors.head()

C:\Users\Pedro\AppData\Local\Temp\ipykernel_1052\1170468790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_directors['id'] = df_directors['id'].astype(int)


,id_movies,credit_id,department,gender,id,job,name,profile_path
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
109,8844,52fe44bfc3a36847f80a7c7d,Directing,2.0,4945,Director,Joe Johnston,/fok4jaO62v5IP6hkpaaAcXuw2H.jpg
122,15602,52fe466a9251416c75077a89,Directing,2.0,26502,Director,Howard Deutch,/68Vae1HkU1NxQZ6KEmuxIpno7c9.jpg
126,31357,52fe44779251416c91011acb,Directing,2.0,2178,Director,Forest Whitaker,/4pMQkelS5lK661m9Kz3oIxLYiyS.jpg
141,11862,52fe44959251416c75039eef,Directing,2.0,56106,Director,Charles Shyer,/hnWGd74CbmTcDCFQiJ8SYLazIXW.jpg


In [ ]:
df_directors.drop(columns=['profile_path'], inplace=True)
df_directors.head()

C:\Users\Pedro\AppData\Local\Temp\ipykernel_1052\3701926937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_directors.drop(columns=['profile_path'], inplace=True)


,id_movies,credit_id,department,gender,id,job,name
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879,Director,John Lasseter
109,8844,52fe44bfc3a36847f80a7c7d,Directing,2.0,4945,Director,Joe Johnston
122,15602,52fe466a9251416c75077a89,Directing,2.0,26502,Director,Howard Deutch
126,31357,52fe44779251416c91011acb,Directing,2.0,2178,Director,Forest Whitaker
141,11862,52fe44959251416c75039eef,Directing,2.0,56106,Director,Charles Shyer


In [ ]:
df_directors.drop(columns=['credit_id'], inplace=True)
df_directors.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49048 entries, 0 to 465084
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id_movies   49048 non-null  int64  
 1   department  49048 non-null  object 
 2   gender      49048 non-null  float64
 3   id          49048 non-null  int64  
 4   job         49048 non-null  object 
 5   name        49048 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 2.6+ MB


C:\Users\Pedro\AppData\Local\Temp\ipykernel_1052\1822706917.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_directors.drop(columns=['credit_id'], inplace=True)


In [67]:
df_directors.to_csv("directors.csv", index=False, encoding='utf-8')